# Side Yard Sun Using Rotating Matrices

In this notebook, I use a vector approach to calculating the angle of the sun in my side yard. I start by calculating the sun's ray at a solar system level (making some simplifying astronomy assumptions), and through several changes to the basis, I zoom into the level of my house. 

In [1]:
import numpy as np

In the base coordinate system $x$ and $y$ are in the earth's orbital plane around the sun with $x$ pointing toward earth from the sun at the winter solstice and $y$ pointing toward earth at the spring equinox. $z$ is the cross product of $x$ and $y$ and is coming straight "up" out of the orbital plane. 

Another way to think of this coordinate system is as fixed to the stars. $x$ is in the direction of the Taurus constellation. $y$ is pointing to Leo.

I'm assuming earth has a circular orbit around the sun which is not quite accurate, but close enough for what I'm calculating.

Let $\theta$ be the angle between the sun to earth ray and $x$.

<img src="solar_coordinates.png" style="width: 400px;">

The direction of the sun's ray onto earth can be represented as the unit vector
$$ \mathbf{r} = \begin{bmatrix}
\cos \theta \\
\sin \theta \\
0 
\end{bmatrix}$$

In [2]:
# define in numpy terms
def ray(theta):
    return np.array([[np.cos(theta)],
                     [np.sin(theta)],
                     [0]])

The first transformation is to convert to the coordinate system in the equatorial plane (imagine an infinite sheet of paper going through the equator). Let's call this orthonormal basis $\mathcal{E}$.

The tilt of the earth is $23.5^{\circ}$.

$$ [\mathbf{r}]_\mathcal{E} = \begin{bmatrix}
\cos 23.5^{\circ} & 0 & -\sin 23.5^{\circ} \\
0 & 1 & 0 \\
\sin 23.5^{\circ} & 0 & \cos 23.5^{\circ}
\end{bmatrix} \mathbf{r}$$

In [3]:
# define in numpy terms

tilt = 23.5*np.pi/180 # converted to radians
E = np.matrix([[np.cos(tilt), 0, -np.sin(tilt)],
               [0,            1, 0],
               [np.sin(tilt), 0, np.cos(tilt)]])

The second transformation is to spin the earth. This basis $\mathcal{Q}$ represents the perspective of earth from a person on the equator. $\mathcal{Q}_x$ is up toward the sky, $\mathcal{Q}_y$ is east, and $\mathcal{Q}_z$ is north.

Let $\phi$ be the angle of rotation of the earth relative to the $x$-axis of $\mathcal{E}$. These earth rotations are measured with reference to the stars, not in reference to the sun as we typically think of rotation.

$$ [\mathbf{r}]_\mathcal{Q} = \begin{bmatrix}
\cos \phi &  \sin \phi & 0\\
-\sin \phi & \cos \phi & 0\\
0 & 0 & 1
\end{bmatrix} [\mathbf{r}]_\mathcal{E}$$



In [4]:
def Q(phi):
    return np.matrix([[np.cos(phi),  np.sin(phi), 0],
                      [-np.sin(phi), np.cos(phi), 0],
                      [0,            0,           1]])

The third transformation is to move from the equator to my latitude with basis $\mathcal{L}$.

The latitude of my house is roughly $33^{\circ}$.

$$ [\mathbf{r}]_\mathcal{L} = \begin{bmatrix}
\cos 33^{\circ} & 0 & \sin 33^{\circ} \\
0 & 1 & 0 \\
-\sin 33^{\circ} & 0 & \cos 33^{\circ}
\end{bmatrix} [\mathbf{r}]_\mathcal{Q}$$

In [5]:
# in numpy terms
latitude = 33*np.pi/180 # converted to radians
L = np.matrix([[np.cos(latitude),  0, np.sin(latitude)],
               [0,                 1, 0],
               [-np.sin(latitude), 0, np.cos(latitude)]])

The final transformation is to orient the coordinates to my side yard fence which runs from northwest to southeast at a $23^{\circ}$ angle with the east-west axis.

<img src="fence_coordinates.png" style="width: 400px;">

The transformation to this final basis $\mathcal{F}$ is 

$$ [\mathbf{r}]_\mathcal{F} = \begin{bmatrix}
1 & 0 & 0 \\
0 & \cos 23^{\circ} & -\sin 23^{\circ} \\
0 & \sin 23^{\circ}  & \cos 23^{\circ} 
\end{bmatrix} [\mathbf{r}]_\mathcal{L}$$

In [6]:
# in terms of numpy
fence_angle = 23*np.pi/180 # convert to radians
F = np.matrix([[1, 0,                   0],
               [0, np.cos(fence_angle), -np.sin(fence_angle)], 
               [0, np.sin(fence_angle), np.cos(fence_angle)]])


Now we're ready to put this all together. In order to get the suns ray in the fence coordinate system, we can string together these matrices. In order to make it all dependent on only one variable, I can substitute $\phi = 366.25 \theta$ because there are 365.25 rotations of the earth each year with reference to the sun--one more rotation when the reference point is the stars. If you're not convinced think about how many times the earth spins if the same side of the earth was always facing the sun. That would be 0 "sun days" and 1 "star day".  

To calculate whether my plants are in shade or direct sunshine, I'll use the $\mathcal{F}_x$ and $\mathcal{F}_z$ components to cast a shadow and check if it covers the entire 6 ft alley. I'm only interested in when $\mathcal{F}_x < 0$ (that is, when it's daytime) and when $\mathcal{F}_z > 0$ since otherwise the plants are shaded by the house.

In [7]:
fence_height_ft = 5.5
alley_length_ft = 6

def daytime(theta):
    fence_ray = F @ L @ Q(366.25*theta) @ E @ ray(theta)
    up = fence_ray[0, 0]
    return up < 0

def in_sun(x, theta):
    fence_ray = F @ L @ Q(366.25*theta) @ E @ ray(theta)
    up = fence_ray[0, 0]
    out = fence_ray[2, 0]
    if up >= 0:
        return False
    if out < 0:
        return False
    shadow = fence_height_ft*out/(-up)
    if shadow > x:
        return False
    else:
        return True
    
def take_sample(fcn, n, subsections):
    sector = 2*np.pi/subsections
    for idx in range(subsections):
        positive_samples = 0
        for theta in np.linspace(idx*sector, (idx+1)*sector - sector/n, n):
            if fcn(theta):
                positive_samples += 1
        print(positive_samples/n)
        
def percentage_daytime(n: int = 10000, subsections: int = 1):
    return take_sample(daytime, n, subsections)
    
def percentage_in_sun(x, n: int = 10000, subsections: int = 1):
    return take_sample(lambda y: in_sun(x, y), n, subsections)

Based on the result below, my plants in the side yard receive direct sunshine 25% of the year capturing half of the sunshine available.

In [8]:
percentage_in_sun(alley_length_ft)

0.2456


In [9]:
percentage_daytime()

0.4999


The summer receives the greater portion of the sunshine when the sun is not so far south. 

In [10]:
percentage_in_sun(alley_length_ft, subsections=4)

0.1676
0.3236
0.3227
0.1688


In [11]:
percentage_daytime(subsections=4)

0.443
0.5574
0.5558
0.4424


On a month by month basis, we see that most of the year has 30-ish% direct sunshine, but around the winter solstice, it drops dramatically down to 5%. These 12 categories don't perfectly match the 12 calendar months of the year, but is rather a even division in 12 parts starting with the winter solstice.

In [12]:
percentage_in_sun(alley_length_ft, subsections=12)

0.0484
0.2037
0.2507
0.3005
0.3362
0.3345
0.333
0.3384
0.2964
0.2519
0.2067
0.0471


The nature of this problem makes rough estimates acceptable, but the estimate could definitely be improved by taking cloud cover into account (although this is suprisingly rare in Southern California).